In [1]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn import cluster
from scipy import stats  ## for z-score. do we still need it?

In [2]:
# Read Scan On Data
scanOn_samp0 = pd.read_csv('all_scanOn_samp0.csv')

scanOn_samp0.head()

,Mode,BusinessDate,DateTime,CardID,CardType,VehicleID,ParentRoute,RouteID,StopID
0,1,2015-07-02,2015-07-02 13:08:13,1524480,9,1222,415,10883,15084
1,3,2015-07-02,2015-07-02 07:33:58,1756270,2,1091,24,15296,18566
2,1,2015-07-02,2015-07-02 16:42:38,10560630,2,1469,862,10227,19824
3,1,2015-07-01,2015-07-01 11:07:08,11812440,2,2886,458,8591,21184
4,1,2015-07-01,2015-07-01 17:28:14,12272500,2,2853,670,16447,21296


In [3]:
for col in scanOn_samp0.columns:
    if (scanOn_samp0[col].dtype.name in ['category', 'str', 'object']):
        # Check for extra whitespaces and remove them
        scanOn_samp0[col] = scanOn_samp0[col].str.strip()

In [4]:
# Read card types
column_names = ("Card_SubType_ID","Card_SubType_Desc","Payment_Type","Fare_Type","Concession_Type","MI_Card_Group")
card_types = pd.read_csv('card_types.txt',sep="|", names=column_names)

card_types.head()

,Card_SubType_ID,Card_SubType_Desc,Payment_Type,Fare_Type,Concession_Type,MI_Card_Group
0,62,Pensioner Concession Card holder PC,Paid,Concession,Other Concession,Other Concession
1,29,Vic HCC - Sickness allowance,Paid,Concession,Other Concession,Other Concession
2,43,First Class Pass,Free,Concession,Free Pass,Other
3,34,PCC - Widow allowance,Paid,Concession,Other Concession,Other Concession
4,47,Victoria Police Travel Authority,Free,Concession,Free Pass,Other


In [5]:
# Read stop locations
column_names = ("StopLocationID","StopNameShort","StopNameLong","StopType","SuburbName","PostCode","RegionName","LocalGovernmentArea","StatDivision","GPSLat","GPSLong")
stop_locations = pd.read_csv('stop_locations.txt',sep="|", names=column_names)

stop_locations.head()

,StopLocationID,StopNameShort,StopNameLong,StopType,SuburbName,PostCode,RegionName,LocalGovernmentArea,StatDivision,GPSLat,GPSLong
0,867,Weemala Court,Weemala Ct/Plenty River Dr (Greensborough),Kerbside,Greensborough,3088.0,Melbourne,Banyule,Greater Metro,-37.689596,145.105088
1,868,Crana Grove,Crana Gr/Plenty River Dr (Greensborough),Kerbside,Greensborough,3088.0,Melbourne,Banyule,Greater Metro,-37.686742,145.105588
2,869,Punkerri Circuit,Punkerri Cct/Plenty River Dr (Greensborough),Kerbside,Greensborough,3088.0,Melbourne,Banyule,Greater Metro,-37.683643,145.108743
3,870,Plenty River Drive,231 Plenty River Dr (Greensborough),Kerbside,Greensborough,3088.0,Melbourne,Banyule,Greater Metro,-37.682591,145.111331
4,875,Oldstead Rd,Oldstead Rd/Diamond Creek Rd (Greensborough),Kerbside,Greensborough,3088.0,Melbourne,Banyule,Greater Metro,-37.685336,145.117319


In [6]:
# Read calendar
column_names = ("CalendarDateSeq","Date","CalendarYear","FinancialYear","FinancialMonth","CalendarMonth","CalendarMonthSeq","CalendarQuarter","FinancialQuarter","CalendarWeek","FinancialWeek","DayType","DayTypeCategory","DayTypeAdditional","WeekdaySeq","WeekDay","FinancialMonthSeq","FinancialMonthName","MonthNumber","ABSWeek","WeekEnding","QuarterName")
calendar = pd.read_csv('calendar.txt',sep="|", names=column_names, encoding='utf-8')
#list(calendar)
calendar.head()

,CalendarDateSeq,Date,CalendarYear,FinancialYear,FinancialMonth,CalendarMonth,CalendarMonthSeq,CalendarQuarter,FinancialQuarter,CalendarWeek,...,DayTypeCategory,DayTypeAdditional,WeekdaySeq,WeekDay,FinancialMonthSeq,FinancialMonthName,MonthNumber,ABSWeek,WeekEnding,QuarterName
0,20170930,2017-09-30,2017,FY2017 - 2018,9,September,201709,2017Q3,FY17-18Q1,39,...,Weekend,Saturday,6,Saturday,201709,Sep 17/,9,1239,w/e 2017-09-30,September Qtr. 2017
1,20080615,2008-06-15,2008,FY2007 - 2008,18,June,200806,2008Q2,FY07-08Q4,24,...,Weekend,Sunday,7,Sunday,200818,Jun 07/,6,755,w/e 2008-06-21,June Qtr. 2008
2,20040222,2004-02-22,2004,FY2003 - 2004,14,February,200402,2004Q1,FY03-04Q3,8,...,Weekend,Normal Sunday,7,Sunday,200414,Feb 03/,2,530,w/e 2004-02-28,March Qtr. 2004
3,20190620,2019-06-20,2019,FY2018 - 2019,18,June,201906,2019Q2,FY18-19Q4,24,...,Weekday,0,4,Thursday,201906,Jun 18/,6,1329,w/e 2019-06-22,June Qtr. 2019
4,20170831,2017-08-31,2017,FY2017 - 2018,8,August,201708,2017Q3,FY17-18Q1,35,...,Weekday,Normal Weekday,4,Thursday,201708,Aug 17/,8,1235,w/e 2017-09-02,September Qtr. 2017


In [7]:
# Some data preperation of scanOn_samp0 data

In [8]:
scanOn_samp0.isnull().sum()

Mode                 0
BusinessDate         0
DateTime             0
CardID               0
CardType             0
VehicleID            0
ParentRoute     993734
RouteID              0
StopID               0
dtype: int64

In [9]:
# Check for missing values and replace with 0
scanOn_samp0.ParentRoute.fillna(0, inplace=True)
print ('Replaced missing values with 0')

Replaced missing values with 0


In [10]:
# Some data prep on calendar data

calendar.WeekEnding = calendar.WeekEnding.str.replace('w/e ', '')
calendar.FinancialMonthName = calendar.FinancialMonthName.str.replace('/','')
calendar.WeekEnding.value_counts()
calendar.FinancialMonthName.value_counts()

calendar.head()

,CalendarDateSeq,Date,CalendarYear,FinancialYear,FinancialMonth,CalendarMonth,CalendarMonthSeq,CalendarQuarter,FinancialQuarter,CalendarWeek,...,DayTypeCategory,DayTypeAdditional,WeekdaySeq,WeekDay,FinancialMonthSeq,FinancialMonthName,MonthNumber,ABSWeek,WeekEnding,QuarterName
0,20170930,2017-09-30,2017,FY2017 - 2018,9,September,201709,2017Q3,FY17-18Q1,39,...,Weekend,Saturday,6,Saturday,201709,Sep 17,9,1239,2017-09-30,September Qtr. 2017
1,20080615,2008-06-15,2008,FY2007 - 2008,18,June,200806,2008Q2,FY07-08Q4,24,...,Weekend,Sunday,7,Sunday,200818,Jun 07,6,755,2008-06-21,June Qtr. 2008
2,20040222,2004-02-22,2004,FY2003 - 2004,14,February,200402,2004Q1,FY03-04Q3,8,...,Weekend,Normal Sunday,7,Sunday,200414,Feb 03,2,530,2004-02-28,March Qtr. 2004
3,20190620,2019-06-20,2019,FY2018 - 2019,18,June,201906,2019Q2,FY18-19Q4,24,...,Weekday,0,4,Thursday,201906,Jun 18,6,1329,2019-06-22,June Qtr. 2019
4,20170831,2017-08-31,2017,FY2017 - 2018,8,August,201708,2017Q3,FY17-18Q1,35,...,Weekday,Normal Weekday,4,Thursday,201708,Aug 17,8,1235,2017-09-02,September Qtr. 2017


In [11]:
# Save dataframes to csv

scanOn_samp0.to_csv('scanOn_samp0_processed.csv', index=False)
card_types.to_csv('card_types_processed.csv', index=False)
stop_locations.to_csv('stop_locations_processed.csv', index=False)
calendar.to_csv('calendar_processed.csv', index=False)

In [12]:
scanOn_samp0.size

14130000

In [89]:
# Read speeds
speeds = pd.read_csv('melbourne_vehicle_traffic.csv',sep=",")

speeds.head()

,location index,lat,lon,mean_speed,std_speed
0,0,-38.100617,145.488326,"[73.82, 82.73, 84.695, 79.11, 76.4425, 66.62, ...","[0.0, 0.0, 5.225000000000001, 11.5499999999999..."
1,1,-38.062604,145.367474,"[56.12142857142857, 56.995000000000005, 70.156...","[11.149351587259527, 23.845000000000002, 9.831..."
2,2,-38.357822,144.757135,"[65.52666666666666, 59.155, 46.449999999999996...","[2.6402819714736676, 2.6219982837522977, 17.92..."
3,3,-38.355186,144.752776,"[76.262, 72.418, 68.33, 73.508, 77.43, 76.47, ...","[1.930444508396961, 2.806730482251543, 4.72478..."
4,4,-38.351950,144.918912,"[58.82400000000001, 55.13249999999999, 50.8799...","[4.580648862333807, 4.693273777439369, 18.3136..."


In [90]:
speeds['week_hour'] = 0

speeds.head()

headers = speeds.dtypes.index

speeds_ = pd.DataFrame(data=None, index=None, columns=headers, dtype=None, copy=False)

speeds_.head()

,location index,lat,lon,mean_speed,std_speed,week_hour


In [91]:
for index, row in speeds.iterrows():
    label = 'mean_speed'
    s = speeds[label].iloc[index]
    s = s[1:(len(s)-1)]
    L = s.split(", ")
    for idx, val in enumerate(L):
        row[3] = val
        row[5] = idx
        #print(row[1])
        speeds_ = speeds_.append(row, ignore_index=True)
        

KeyboardInterrupt: 

In [92]:
speeds_.head()

,location index,lat,lon,mean_speed,std_speed,week_hour
0,0,-38.100617,145.488326,73.82,"[0.0, 0.0, 5.225000000000001, 11.5499999999999...",0
1,0,-38.100617,145.488326,82.73,"[0.0, 0.0, 5.225000000000001, 11.5499999999999...",1
2,0,-38.100617,145.488326,84.695,"[0.0, 0.0, 5.225000000000001, 11.5499999999999...",2
3,0,-38.100617,145.488326,79.11,"[0.0, 0.0, 5.225000000000001, 11.5499999999999...",3
4,0,-38.100617,145.488326,76.4425,"[0.0, 0.0, 5.225000000000001, 11.5499999999999...",4


In [93]:
#speeds['mean_speed'] = speeds['mean_speed'].map(lambda x: x.lstrip('[').rstrip(']'))
#speeds['std_speed'] = speeds['std_speed'].map(lambda x: x.lstrip('[').rstrip(']'))
speeds['mean_speed'] = speeds['mean_speed'].str.replace('[', '')
speeds['mean_speed'] = speeds['mean_speed'].str.replace(']', '')

In [94]:
speeds['std_speed'] = speeds['std_speed'].str.replace('[', '')
speeds['std_speed'] = speeds['std_speed'].str.replace(']', '')

In [95]:
speeds.head()

,location index,lat,lon,mean_speed,std_speed,week_hour
0,0,-38.100617,145.488326,"73.82, 82.73, 84.695, 79.11, 76.4425, 66.62, 5...","0.0, 0.0, 5.225000000000001, 11.54999999999999...",0
1,1,-38.062604,145.367474,"56.12142857142857, 56.995000000000005, 70.1566...","11.149351587259527, 23.845000000000002, 9.8312...",0
2,2,-38.357822,144.757135,"65.52666666666666, 59.155, 46.449999999999996,...","2.6402819714736676, 2.6219982837522977, 17.928...",0
3,3,-38.355186,144.752776,"76.262, 72.418, 68.33, 73.508, 77.43, 76.47, 7...","1.930444508396961, 2.806730482251543, 4.724785...",0
4,4,-38.351950,144.918912,"58.82400000000001, 55.13249999999999, 50.87999...","4.580648862333807, 4.693273777439369, 18.31367...",0


In [98]:
speeds_.to_csv('speeds_processed2.csv', index=False)